In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
their_df = pd.read_excel('data/team5_final_data.xlsx')
our_df = pd.read_pickle('data/ours.bin')

In [3]:
their_df.head(1)

,Unnamed: 0,image_link,temperature,description,logits,hallucinations,hedges,probe_1,label_1,pred_1,...,pred_3,context_3,probe_4,label_4,pred_4,context_4,pred_1_prob,pred_2_prob,pred_3_prob,pred_4_prob
0,0,https://cdn.pixabay.com/photo/2021/12/18/06/01...,0.8,The image depicts a lively street scene with ...,"[(' The', {'The': 0.9288, 'This': 0.0377, 'In'...",The image depicts a lively street scene with c...,The image depicts a lively street scene with c...,All buildings are painted white.,False,False,...,False,The image depicts a lively street scene with c...,There is only cars in the street.,False,False,The image depicts a lively street scene with c...,0.76908,0.566019,0.715424,0.727975


In [4]:
our_df.head(1)

,text,generated_logits,hallucinations,hedges,probe_1,label_1,pred_1,context_1,probe_2,label_2,...,context_2,probe_3,label_3,pred_3,context_3,probe_4,label_4,pred_4,context_4,hillucination_text
https://cdn.pixabay.com/photo/2018/03/30/22/05/teamwork-3276682_960_720.jpg,"<0x0A>In the center of the image, there are th...","[(<0x0A>, {'<0x0A>': 0.7763559818267822, ' I':...","<0x0A>In the center of the image, there are [t...","<0x0A>In the center of the image, there are th...",puzzle pieces on a light-colored background,True,"{' No': 0.18142198026180267, ' Yes': 0.8130769...","<0x0A>In the center of the image, there are th...",three-dimensional effect,False,...,"<0x0A>In the center of the image, there are th...",The face appears to be smiling and looking tow...,True,"{' No': 0.27034449577331543, ' Yes': 0.7234793...","<0x0A>In the center of the image, there are th...","On the left side of the puzzle, there is a blue",True,"{' No': 0.3292682468891144, ' Yes': 0.66514015...","<0x0A>In the center of the image, there are th...",NaN


In [5]:
print(f'their columns: {their_df.columns}')
print(f'our columns: {our_df.columns}')

their columns: Index(['Unnamed: 0', 'image_link', 'temperature', 'description', 'logits',
       'hallucinations', 'hedges', 'probe_1', 'label_1', 'pred_1', 'context_1',
       'probe_2', 'label_2', 'pred_2', 'context_2', 'probe_3', 'label_3',
       'pred_3', 'context_3', 'probe_4', 'label_4', 'pred_4', 'context_4',
       'pred_1_prob', 'pred_2_prob', 'pred_3_prob', 'pred_4_prob'],
      dtype='object')
our columns: Index(['text', 'generated_logits', 'hallucinations', 'hedges', 'probe_1',
       'label_1', 'pred_1', 'context_1', 'probe_2', 'label_2', 'pred_2',
       'context_2', 'probe_3', 'label_3', 'pred_3', 'context_3', 'probe_4',
       'label_4', 'pred_4', 'context_4', 'hillucination_text'],
      dtype='object')


In [6]:
our_df.rename(columns={
    'text': 'description',
    'generated_logits': 'logits',
}, inplace=True)
our_df['temperature'] = 0.7
our_df.drop(columns=['hillucination_text'], inplace=True)

their_df.drop(columns=['Unnamed: 0'], inplace=True)
# our_df's index is the image_link, i want to make it the same as their_df as a column
our_df.reset_index(inplace=True)
our_df.rename(columns={'index': 'image_link'}, inplace=True)

for i in range(1, 5):
    their_df[f'pred_{i}'] = their_df[f'pred_{i}_prob']
    # drop the _prob columns
    their_df.drop(columns=[f'pred_{i}_prob'], inplace=True)
    # normalize the probabilities so it matches their format
    our_df[f'pred_{i}'] = our_df[f'pred_{i}'].apply(lambda x: x[' Yes'] / (x[' Yes'] + x[' No']))
    

In [7]:
our_df = our_df[their_df.columns]

In [8]:
print(f'their columns: {their_df.columns}')
print(f'our columns: {our_df.columns}')

their columns: Index(['image_link', 'temperature', 'description', 'logits', 'hallucinations',
       'hedges', 'probe_1', 'label_1', 'pred_1', 'context_1', 'probe_2',
       'label_2', 'pred_2', 'context_2', 'probe_3', 'label_3', 'pred_3',
       'context_3', 'probe_4', 'label_4', 'pred_4', 'context_4'],
      dtype='object')
our columns: Index(['image_link', 'temperature', 'description', 'logits', 'hallucinations',
       'hedges', 'probe_1', 'label_1', 'pred_1', 'context_1', 'probe_2',
       'label_2', 'pred_2', 'context_2', 'probe_3', 'label_3', 'pred_3',
       'context_3', 'probe_4', 'label_4', 'pred_4', 'context_4'],
      dtype='object')


In [9]:
their_df['group_num'] = 5
our_df['group_num'] = 4

In [10]:
united_df = pd.concat([their_df, our_df], ignore_index=True)

In [11]:
united_df.to_csv('data/united.csv', index=False)

In [13]:
united_df.head(1)

,image_link,temperature,description,logits,hallucinations,hedges,probe_1,label_1,pred_1,context_1,...,context_2,probe_3,label_3,pred_3,context_3,probe_4,label_4,pred_4,context_4,group_num
0,https://cdn.pixabay.com/photo/2021/12/18/06/01...,0.8,The image depicts a lively street scene with ...,"[(' The', {'The': 0.9288, 'This': 0.0377, 'In'...",The image depicts a lively street scene with c...,The image depicts a lively street scene with c...,All buildings are painted white.,False,0.76908,The image depicts a lively street scene [with ...,...,The image depicts a lively street scene with c...,There are pedastrians walking.,True,0.715424,The image depicts a lively street scene with c...,There is only cars in the street.,False,0.727975,The image depicts a lively street scene with c...,5


In [14]:
united_df.tail(1)

,image_link,temperature,description,logits,hallucinations,hedges,probe_1,label_1,pred_1,context_1,...,context_2,probe_3,label_3,pred_3,context_3,probe_4,label_4,pred_4,context_4,group_num
349,https://avatars.mds.yandex.net/i?id=58a0836f7d...,0.7,The image shows three siamese kittens sitting...,"[( The, {'<0x0A>': 0.7223991751670837, ' I': 0...",The image shows [three] siamese kittens sitti...,The image shows three siamese kittens sitting...,The image shows three siamese kittens,False,0.73412,[The image shows three siamese kittens] sitti...,...,The image shows three siamese kittens sitting...,The kittens have striking blue eyes,True,0.672332,The image shows three siamese kittens sitting...,"They appear to be young, with a light cream co...",True,0.672332,The image shows three siamese kittens sitting...,4
